Необходимо установить git: https://git-scm.com/. Стандартные настройки подойдут. 

Следующие модули лучше устанавливать именно в таком порядке (rnnmorph и PyKoSpacing иба используют tensorflow и keras, и rnnmorph может поставить автоматически более новые версии, которые не работают с pykospacing)

pip install razdel

pip install rnnmorph

pip install spacy_udpipe 

pip install keras==2.4.3

pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [1]:
'''Модули, которые необходимо импортировать. 
PyKoSpacing - токенизатор корейского; spacy_udpipe - морфопарсер (мы загрузим его для корейского); 
razdel - токенизатор для русского; rnnmorph - морфопарсер русского'''

import json
import re
import pandas as pd
from pykospacing import Spacing
from razdel import tokenize
from rnnmorph.predictor import RNNMorphPredictor
import spacy_udpipe

In [2]:
def loader(path, lang):
    '''Функция загрузки данных'''
    data = []
    with open(path, 'r', encoding='utf8') as file:  
        for line in file:
            data.append(json.loads(line))
    data = pd.DataFrame(data)
    data = data[['tweet', 'language']]  # оставим в датафрейме только нужные колонки
    data = data[data.language == lang]  # отсеем твиты, которые сам твиттер определил как не на нужном нам языке
    data['tweet'] = data['tweet'].apply(lambda tweet: re.sub(r'https://t.co/\w+', '', tweet))  # удалим ссылки. по-хорошему, там еще надо чистить, но нам не так принципиально
    data['tweet'] = data['tweet'].apply(lambda tweet: re.sub(r'@[a-zA-Z0-9_-]+', '', tweet))
    return data

In [3]:
'''Загрузим датасет и переведем его в формат panel datas'''

korean = loader('korean.json', 'ko')

korean.head()

,tweet,language
0,류짱우 겨 존나야함,ko
1,유진차 ㅜㅋㅋㅋㅋㅋㅋㅋㅋㅋ,ko
2,찰떡같은거 나올때까지 시도했기때문이긴한데;; 아무튼 소름 ㄷㄷ;;,ko
3,ㅈㄴ찰떡같은거만 나오네;;,ko
4,🐞 수인 테스트 ────────────────── 김래빈님은 자신이 토끼 수인인 걸...,ko


In [4]:
'''Загрузим необходимые для работы инструменты - модель для корейского'''

spacy_udpipe.download("ko")
nlp = spacy_udpipe.load("ko")
spacing = Spacing()

Already downloaded a model for the 'ko' language


Примечание: ячейки с кодом для токенизации и морфоразбора могут **долго** работать. Это нормально: почти все наши инструменты работают на нейронных сетях, а они небыстрые.

In [5]:
'''Отпарсим твиты: токенизируем их и сделаем морфоразбор. Теперь у нас будут списки токенов и лемм'''

parsedkorean = []
for tweet in korean.tweet:
    doc = nlp(spacing(tweet))
    parsedtweet = []
    for token in doc:
        parsedtweet.append((token.text, token.lemma_))
    parsedkorean.append(parsedtweet)

In [6]:
'''Можно посмотреть, как это будет выглядеть, на первых трех предложениях'''
parsedkorean[:3]

[[('류짱우', '류짱우'), ('겨', '기+어'), ('존', '존'), ('나야함', '나야+하+ㅁ')],
 [('유진차', '유진차'), ('ㅜㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'ㅜㅋㅋㅋㅋㅋㅋㅋㅋㅋ')],
 [('찰떡', '찰떡'),
  ('같은', '같+ㄴ'),
  ('거', '거'),
  ('나올', '나오+ㄹ'),
  ('때까지', '때+까지'),
  ('시도', '시+도'),
  ('했기', '하+었+기'),
  ('때문이긴', '때문+이+기+는'),
  ('한데', '한데'),
  (';', ';'),
  (';', ';'),
  ('아무튼', '아무튼'),
  ('소름', '소름'),
  ('ㄷㄷ', 'ㄷㄷ'),
  (';', ';'),
  (';', ';')]]

In [7]:
'''Зададим слово, предложения с которым мы хотим отсеять'''

keyword = '것'  # меняйте на нужное слово. Это может быть или токен, или лемма (но я не знаю, как в корейском языке достоверно лемматизируется, есть ли у них инфинитивы?)
filteredkorean = []
for tweet in parsedkorean:
    for token in tweet:
        if keyword in token:
            filteredkorean.append(tweet)
            break
print(len(filteredkorean))  # посмотрим, сколько предложений у нас получилось

117


In [8]:
'''Запишем предложения в xlsx'''

d = dict(enumerate(filteredkorean))
dict_df = pd.DataFrame({key:pd.Series(value) for key, value in d.items()}).T
dict_df.to_excel('korean.xlsx')  # Меняйте название файла

Теперь то же сделаем для русского языка

In [9]:
'''Инициализируем морфопарсер - если вывалит какие-то ворнинги, не пугайтесь, он любит. Главное, чтобы не было ошибок в нижних ячейках'''

predictor = RNNMorphPredictor(language="ru")

In [10]:
russian = loader('russian.json', 'ru')
russian.head()

,tweet,language
0,Сама 10 мес искала работу. Знаю.,ru
1,Железный аргумент.,ru
2,Т.е. назло маме отморожу уши?,ru
3,Это его выбор.,ru
4,Присоединяйтесь ко мне в моей Аудиокомнате!,ru


Будьте готовы к тому, что RNNMorph **очень** медленный. Можно идти пить чай...

In [11]:
parsedrussian = []
for tweet in russian.tweet:
    parsedrussian.append([(w.word, w.normal_form) for w in predictor.predict([t.text for t in tokenize(tweet)])])

In [12]:
parsedrussian[:3]

[[('Сама', 'сам'),
  ('10', '10'),
  ('мес', 'мес'),
  ('искала', 'искать'),
  ('работу', 'работа'),
  ('.', '.'),
  ('Знаю', 'знать'),
  ('.', '.')],
 [('Железный', 'железный'), ('аргумент', 'аргумент'), ('.', '.')],
 [('Т', 'так'),
  ('.', '.'),
  ('е', 'быть'),
  ('.', '.'),
  ('назло', 'назло'),
  ('маме', 'мама'),
  ('отморожу', 'отморозить'),
  ('уши', 'ухо'),
  ('?', '?')]]

In [13]:
'''Зададим слово, предложения с которым мы хотим отсеять'''

keyword = 'делать'  # меняйте на нужное слово. Это может быть или токен, или лемма (для русского вполне можно использовать инфинитивы)
filteredrussian = []
for tweet in parsedrussian:
    for token in tweet:
        if keyword in token:
            filteredrussian.append(tweet)
            break
print(len(filteredrussian))  # посмотрим, сколько предложений у нас получилось

39


In [14]:
'''Запишем предложения в xlsx'''

d = dict(enumerate(filteredrussian))
dict_df = pd.DataFrame({key:pd.Series(value) for key, value in d.items()}).T
dict_df.to_excel('russian.xlsx')  # Меняйте название файла